In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import random
from selenium.webdriver.support import expected_conditions as EC
import time
from random import randint
from thefuzz import process
import json
from fuzzywuzzy import process, fuzz
import requests
# Set the path to your downloaded Edge WebDriver executable

driver = webdriver.Edge()

# Initialize the Edge WebDriver
# driver = webdriver.Edge(service=edge_service)
# driver.implicitly_wait(randint(1,3))



c:\Users\localadmin\Desktop\myprojects\linkedinsucks\linkedinbot_venv\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#list all paths here 
secrets_file = r"C:\Users\localadmin\Desktop\myprojects\linkedinsucks\secrets.json"
mydata_path = r"C:\Users\localadmin\Desktop\myprojects\linkedinsucks\linkedinsucks\mydata"
resume_path = r"C:\Users\localadmin\Desktop\myprojects\linkedinsucks\linkedinsucks\resume\resume.pdf"
questions_file = mydata_path+'\\questions.json'

In [3]:
def read_json(json_path):
    with open(json_path,  'r') as file:
        data = json.load(file)
    return data

def write_json(json_data, json_path):
    with open(json_path , 'w') as file:
        json.dump(json_data, file, indent=4)
    file.close()

In [4]:
secrets = read_json(secrets_file)

In [5]:
#user details
# define system variables for the url 


linkedin_url = "https://www.linkedin.com/"
password = secrets['linkedin_password']
username = secrets['linkedin_loginid']
role = "accountant"
location = "United States"

In [6]:
def open_linkedin_and_signin():
    driver.get(linkedin_url)
    driver.maximize_window()
    signin_page=driver.find_element("xpath", ".//a[contains(text(), 'Sign in') and @href]")
    signin_page.click()
    username_field = driver.find_element("xpath" , ".//input[@id = 'username']")
    password_field = driver.find_element("xpath" , ".//input[@id = 'password']")
    submit_button = driver.find_element("xpath" , ".//button[@type = 'submit']")
    username_field.send_keys(username)
    password_field.send_keys(password)
    submit_button.click()

In [7]:
# defining job search filters  

def easy_apply():
    easy_apply_button = driver.find_element( "xpath" , "//button[contains(@aria-label ,'Easy Apply filter.')]") 
    easy_apply_button.click()

In [8]:
#opening linkedin 
def get_url(url):
    driver.get(url)
    driver.maximize_window()
    
    
def sign_in(username, password):
    signin_page = driver.find_element("xpath", ".//a[contains(text(), 'Sign in') and @href]")
    signin_page.click()
    username_field = driver.find_element("xpath" , ".//input[@id = 'username']")
    password_field = driver.find_element("xpath" , ".//input[@id = 'password']")
    submit_button = driver.find_element("xpath" , ".//button[@type = 'submit']")
    
    username_field.send_keys(username)
    password_field.send_keys(password)
    submit_button.click()

    
def job_search_and_filters():
    get_url("https://www.linkedin.com/jobs")
    role_input = driver.find_element("xpath" , ".//input[@class = 'jobs-search-box__text-input jobs-search-box__keyboard-text-input jobs-search-global-typeahead__input']")
    role_input.send_keys(role)
    time.sleep(1)
    location_input = driver.find_element("xpath" , ".//input[@aria-label = 'City, state, or zip code' and @autocomplete=  'address-level2']")
    location_input.send_keys(location)
    #submit
    role_input.send_keys(Keys.ENTER)
    time.sleep(2)
    easy_apply()

In [9]:
time_dict = {
    "Any time": 0,
    "Past month": 1,
    "Past week": 2,
    "Past 24 hours": 3
}


exp_dict = {
    "Internship": 0,
    "Entry level": 1,
    "Associate": 2,
    "Mid-Senior level": 3,
    "Director": 4,
    "Executive": 5
}

salary_dict = {
    "$40,000+": 0,
    "$60,000+": 1,
    "$80,000+": 2,
    "$100,000+": 3,
    "$120,000+": 4,
    "$140,000+": 5,
    "$160,000+": 6,
    "$180,000+": 7,
    "$200,000+": 8
}

workmode_dict = {
    "On-site": 0,
    "Remote": 1,
    "Hybrid": 2
}


In [10]:
def check_no_matches():
    no_matches  = driver.find_elements('xpath' , ".//h1[@class='t-24 t-black t-normal text-align-center']")
    if no_matches:
        return True
def reset_filters():
    reset_button = driver.find_element('xpath',".//button[@aria-label='Reset applied filters']")
    reset_button.click()



In [11]:
def date_filter( user_date):
    date_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'timePostedRange']  " )
    date_options = date_button.find_elements( "xpath" , ".//li[@class = 'search-reusables__collection-values-item'] ")
    date_button.click()
    label = date_options[time_dict[user_date]].find_element("xpath" , ".//label")
    label.click()
    results = date_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()



def exp_filter( exp_selected):
    exp_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'experience']  " )
    exp_options = exp_button.find_elements("xpath" , ".//li[@class = 'search-reusables__collection-values-item'] ")
    exp_button.click()
    for exp in exp_selected:
        label = exp_options[exp_dict[exp]].find_element("xpath" , ".//label")
        label.click()
    results = exp_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()


def salary_filter(  base_salary):
    salary_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'salaryBucketV2']  " )
    salary_options = salary_button.find_elements("xpath" , ".//li[@class = 'search-reusables__collection-values-item'] ")
    salary_button.click()
    label = salary_options[salary_dict[base_salary]].find_element("xpath" , ".//label")
    label.click()
    results = salary_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()


def workmode_filter(  workmode_selected):
    workmode_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'workplaceType']  " )
    hybrid_options = workmode_button.find_elements("xpath" , ".//li[@class = 'search-reusables__collection-values-item'] ")
    workmode_button.click()
    for mode in workmode_selected:
        label = hybrid_options[workmode_dict[mode]].find_element("xpath" , ".//label")
        label.click()
    results = workmode_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()




def company_filter(companies):

    company_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'company']  ")
    company_button.click()
    input_field = driver.find_element("xpath", ".//input[@aria-label='Add a company']")
    for company in companies:
    
        # input_field = driver.find_element("xpath", ".//input[@aria-label='Add a company']")
        input_field.clear()
        input_field.send_keys(f"{company}")
        time.sleep(0.5)
        input_field.send_keys(Keys.ARROW_DOWN)
        input_field.send_keys(Keys.ENTER)
    results = company_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()



In [12]:
# date_params = 'Past month'
# exp_param = ['Internship' , 'Entry level']
# salary_param = "$80,000+"
# workmode_param= ['On-site' , 'Hybrid']
# companies = ['blackrock' , 'amazon']

In [13]:
# date_filter(date_params)
# exp_filter(exp_param)
# salary_filter(salary_param)
# workmode_filter(workmode_param)
# company_filter(companies=companies)

In [14]:
def theresubmit(driver):
    buttons = driver.find_elements( "xpath" , ".//button[@aria-label = 'Submit application']") 
    return len(buttons) and True

def get_submit(driver):
    button = driver.find_element( "xpath" , ".//button[@aria-label = 'Submit application']") 
    button.click()



def there_resume(form):
    resumes = form.find_elements("xpath" ,".//div[contains(@class, 'ui-attachment jobs-document-upload-redesign-card__container')]")
    return len(resumes) and True
    

def select_resume( number):
    resumes = driver.find_elements("xpath" ,".//div[contains(@class, 'ui-attachment jobs-document-upload-redesign-card__container')]")
    driver.execute_script("arguments[0].click();", resumes[number]) 
    resumes[number].click()   
    


def click_next():
    button = driver.find_element("xpath" , ".//button[@aria-label = 'Continue to next step' or @aria-label = 'Review your application']")
    button.click()


In [15]:


# Load the Q&A dictionary
qna_dict = read_json(questions_file)

def rag_query(query):
    url = "http://127.0.0.1:8000/resume_qa"
    payload = {"query": query}
    response = requests.post(url, json=payload)
    return response.json().get('response')

def qna_engine(question):
    question = str(question)
    questions_list = list(qna_dict.keys())

    if question in qna_dict:
        return qna_dict[question]

    threshold = 85
    matches = process.extract(question, questions_list, scorer=fuzz.WRatio, limit=3)
    filtered_matches = [match for match in matches if match[1] >= threshold]

    if filtered_matches:
        return qna_dict[filtered_matches[0][0]]

    return rag_query(question)


In [27]:
def radio_select(radio_buttons):

    for radio_button in radio_buttons:


        question = radio_button.find_element(By.XPATH, ".//legend/span[@data-test-form-builder-radio-button-form-component__title]").text
        option_elements = radio_button.find_elements(By.XPATH, ".//div[@data-test-text-selectable-option]")
        options = []
        
        for option in option_elements:
            options.append(  option.find_element(By.XPATH, ".//label").text)
        print(question, options)
        answer = qna_engine(question)
        
        answer_match = process.extractOne(answer, options)[0] 

        
        selected_option = option_elements[options.index(answer_match)].find_element(By.XPATH, ".//input[@type='radio']")
        driver.execute_script("arguments[0].click();", selected_option)
        #driver.execute_script("arguments[0].click();", selected_option)
    

def dropdown_select(dropdowns):
    # Extract the question
    for dropdown_element in dropdowns:
        question = dropdown_element.find_element(By.XPATH, ".//label/span[@aria-hidden='true']").text
        
        # Extract the options
        options = [option.text for option in dropdown_element.find_elements(By.XPATH, ".//select/option")]
        
        # Print the question and options
        print("Question:", question)
        print("Options:", options)
        answer = qna_engine(question)
        # Find the best match for the answer
        answer_match = process.extractOne(answer, options)[0]
        
        # Select the matched option using Select
        select_element = dropdown_element.find_element(By.XPATH, ".//select[contains(@id, 'text-entity-list-form-component-formElement')]")
        select = Select(select_element)
        select.select_by_visible_text(answer_match)





def fill_single_line_text_field(text_fields):
    for text_field in text_fields:
        # Extract the question
        question = text_field.find_element(By.XPATH, ".//label").text
        
        # Print the question
        print("Question:", question)
        answer = qna_engine(question)
        
        # Find the input field and clear any existing text
        input_field = text_field.find_element(By.XPATH, ".//input[@type='text']")
        input_field.clear()  # Clear the existing input if any
        input_field.send_keys(answer)  # Fill the answer in the input field

def fill_multiline_text_field(text_fields):
    for text_field in text_fields:
        # Extract the question
        question = text_field.find_element(By.XPATH, ".//label").text
        answer = qna_engine(question)
        
        # Print the question
        print("Question:", question)
        
        # Find the textarea field and clear any existing text
        textarea_field = text_field.find_element(By.XPATH, ".//textarea")
        textarea_field.clear()  # Clear the existing input if any
        textarea_field.send_keys(answer)  # Fill the answer in the textarea field







def checkbox_select(fieldset_elements):
    # Extract the question
    for fieldset_element in fieldset_elements:
        question = fieldset_element.find_element(By.XPATH, ".//legend/div[@data-test-checkbox-form-title='true']").text
        answers = qna_engine(question)
        # Extract the options
        option_elements = fieldset_element.find_elements(By.XPATH, ".//div[@data-test-text-selectable-option]")
        options = [option.find_element(By.XPATH, ".//label").text for option in option_elements]
        
        # Print the question and options
        print("Question:", question)
        print("Options:", options)
        
        # Select the matched options
        for answer in answers:
            answer_match = process.extractOne(answer, options)[0]
            option_index = options.index(answer_match)
            checkbox = option_elements[option_index].find_element(By.XPATH, ".//input[@type='checkbox']")
            if not checkbox.is_selected():
                checkbox.click()



def autofill_answer(autofill_fields):
    for autofill_field in autofill_fields:
        # Extract the question
        question = autofill_field.find_element(By.XPATH, ".//label").text
        print("Question:", question)
        
        
        # Find the input field
        input_field = autofill_field.find_element(By.XPATH, ".//input[@aria-autocomplete='list']")
        
        # Get the answer from the user
        answer  = qna_engine(question)
        
        # Fill the answer in the input field
        input_field.send_keys(answer)
        time.sleep(1.0)
        input_field.send_keys(Keys.DOWN)
        input_field.send_keys(Keys.ENTER)

    

In [26]:
def fill_typeahead_input(form_element, answer):
    # Extract the question
    question = form_element.find_element(By.XPATH, ".//label").text
    
    # Print the question
    print("Question:", question)
    
    # Fill the answer in the input field
    input_field = form_element.find_element(By.XPATH, ".//input[@type='text']")
    input_field.send_keys(answer)

In [17]:
def apply_job(job_card):
    job_card.click()
    try:
        easy_apply_button = driver.find_element("xpath" , ".//button[contains(@aria-label , 'Easy Apply to')]")
        easy_apply_button.click()
    except Exception as e:
        return
    print("clicked easy apply")
    
    filledresume = False
    applying = True

    while applying:
        if theresubmit(driver):
            get_submit(driver)
            break

        
        apply = driver.find_element("xpath" , ".//div[@class='jobs-easy-apply-content']")
        form = apply.find_element("xpath", ".//form")

        if not filledresume and there_resume(form):
            select_resume(0)
            filledresume = True

        form = driver.find_element("xpath", ".//form")
        radios = form.find_elements("xpath" , ".//fieldset[@data-test-form-builder-radio-button-form-component = 'true']" )
        if radios:
            radio_select(radios )

        form = driver.find_element("xpath", ".//form")
        text_dropdowns = form.find_elements("xpath" ,".//div[@data-test-text-entity-list-form-component='']")
        if text_dropdowns:
            dropdown_select(text_dropdowns)

        form = driver.find_element("xpath", ".//form")
        singleline_textfields = form.find_elements("xpath" , ".//div[@data-test-single-line-text-form-component='']")
        if singleline_textfields:
            fill_single_line_text_field(singleline_textfields)

        form = driver.find_element("xpath", ".//form")
        multiline_fields = form.find_elements("xpath" , ".//div[@data-test-multiline-text-form-component='']")
        if multiline_fields:
            fill_multiline_text_field(multiline_fields)

        form = driver.find_element("xpath", ".//form")
        checkboxes = form.find_elements("xpath" , ".//fieldset[@data-test-checkbox-form-component='true']") 
        if checkboxes:
            checkbox_select(checkboxes)
        autofills =  form.find_elements("xpath" , ".//div[@data-test-single-typeahead-entity-form-component='']")
        if autofills:
            autofill_answer(autofills)

        click_next()
        


    

    time.sleep(2)
    closingpopup = True
    while closingpopup: 
        
        print("finding exit")
        exit_buttons = driver.find_elements("xpath" , ".//button[@aria-label='Dismiss' ]")
        if exit_buttons:
            exit_buttons = driver.find_elements("xpath" , ".//button[@aria-label='Dismiss' ]")
            exit_buttons[0].click()
            print("exited")
            return
        


In [18]:
# def apply_job(job_card):
    
#     job_card.click()

    
#     easy_apply_button = driver.find_element("xpath", ".//button[contains(@aria-label , 'Easy Apply to')]")
#     easy_apply_button.click()
#     print("clicked easy apply")
    
#     isform = True
#     fillresume = False
    
#     while isform:

#         if theresubmit(driver):
#             get_submit(driver)


        
#         apply = driver.find_element("xpath" , ".//div[@class='jobs-easy-apply-content']")
#         try:
#             form = apply.find_element("xpath", ".//form")
#         except Exception as e:
#             isform = False
        
#         if not fillresume:
            
#             if there_resume(form):
#                 print("reached resume")
#                 resumes = form.find_elements("xpath" ,".//div[contains(@class, 'ui-attachment jobs-document-upload-redesign-card__container')]")
#                 driver.execute_script("arguments[0].click();", resumes[0]) 
#                 resumes[0].click()
#                 # enter_resume(form, 0)
#                 fillresume = True

          
#         fields = form.find_elements("xpath" , "//div[contains(@class, 'jobs-easy-apply-form-section__grouping')]")
#         radios = form.find_elements("xpath" , ".//fieldset[@data-test-form-builder-radio-button-form-component = 'true']" )
#         text_dropdowns = form.find_elements("xpath" ,".//div[@data-test-text-entity-list-form-component='']")
#         singleline_textfields = form.find_elements("xpath" , ".//div[@data-test-single-line-text-form-component='']")
#         multiline_fields = form.find_elements("xpath" , ".//div[@data-test-multiline-text-form-component='']")
#         checkboxes = form.find_elements("xpath" , ".//fieldset[@data-test-checkbox-form-component='true']") 
#         autofills = form.find_elements("xpath" , ".//input[@aria-autocomplete='list']")

#         for radio in radios:
#             radio_select(radio)
#         for textdropdown in text_dropdowns:
#             dropdown_select(textdropdown)
#         for singletext in singleline_textfields:
#             fill_single_line_text_field(singletext)
#         for multitext in multiline_fields:
#             fill_multiline_text_field(multitext)
#         for checkbox in checkboxes:
#             checkbox_select(checkbox)
#         for autofill in autofills:
#             autofill_answer(autofill)
      
        



#         find_next(form)

        
 


    
#     exit_buttons = driver.find_elements("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view mlA block' and .//span[text()='Done' ]]")
#     print("reacjhed exot")
#     if exit_buttons:
#         exit_buttons[0].click()

    


In [19]:
def iterator( driver):
    currentpage = 1
    nextpageexists = True
    
    while nextpageexists:
        
        search_results = driver.find_element("xpath" , ".//div[contains(@class, 'jobs-search-results-list')]")
        list_of_jobs = search_results.find_elements( "xpath" , "//li[@data-occludable-job-id]")
        
        attempt_job = 0
        while attempt_job < 3:
            try:
                driver.execute_script("arguments[0].scrollIntoView();", list_of_jobs[-1])
                
                attempt_job = 4
            except Exception as e:
                attempt_job += 1
        if attempt_job == 3:
            return
       
        for job in list_of_jobs:
            time.sleep(1.0)
            job.click() 
            time.sleep(0.2)
            apply_job(job)
            write_json(qna_dict, questions_file)
            
        
        

        
        attempt_job = 0
        while attempt_job < 3:
            try:
                
                driver.execute_script("arguments[0].click();", list_of_jobs[-1])
                attempt_job = 4
            except Exception as e:
                attempt_job += 1
        if attempt_job == 3:
            return
        
        # next_pages = driver.find_elements("xpath", f"//button[@aria-label = 'Page {currentpage + 1}']")
        currentpage+=1


        attempts = 0
        while attempts < 5:
            try:
                driver.find_element("xpath", f"//button[@aria-label = 'Page {currentpage + 1}']").click()
                nextpageexists = True
                attempts = 5
            except Exception as e:
                attempts += 1
                nextpageexists = False

        if nextpageexists:
            print("moving to next page")
            
            
        else:
            print("ending function")
            return
    return
    

            

In [20]:
open_linkedin_and_signin()
job_search_and_filters()

In [25]:

iterator(driver)

clicked easy apply
Question: Email address
Options: ['Select an option', 'ganeshrajk007@gmail.com', 'ganesh_012@outlook.com']
Question: Phone country code
Options: ['Select an option', 'Guam (+1)', 'United States (+1)', 'Afghanistan (+93)', 'Aland Islands (+358)', 'Albania (+355)', 'Algeria (+213)', 'American Samoa (+1)', 'Andorra (+376)', 'Angola (+244)', 'Anguilla (+1)', 'Antarctica (+0)', 'Antigua and Barbuda (+1)', 'Argentina (+54)', 'Armenia (+374)', 'Aruba (+297)', 'Australia (+61)', 'Austria (+43)', 'Azerbaijan (+994)', 'Bahamas (+1)', 'Bahrain (+973)', 'Bangladesh (+880)', 'Barbados (+1)', 'Belarus (+375)', 'Belgium (+32)', 'Belize (+501)', 'Benin (+229)', 'Bermuda (+1)', 'Bhutan (+975)', 'Bolivia (+591)', 'Bosnia and Herzegovina (+387)', 'Botswana (+267)', 'Bouvet Island (+0)', 'Brazil (+55)', 'British Indian Ocean Territory (+246)', 'Brunei Darussalam (+673)', 'Bulgaria (+359)', 'Burkina Faso (+226)', 'Burundi (+257)', 'Cambodia (+855)', 'Cameroon (+237)', 'Canada (+1)', 'Cap

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//input[@aria-autocomplete='list']"}
  (Session info: MicrosoftEdge=128.0.2739.42); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF69F2A3A12+10594]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF69F240439+323321]
	(No symbol) [0x00007FF69F05F16E]
	(No symbol) [0x00007FF69F0A656E]
	(No symbol) [0x00007FF69F0A65F5]
	(No symbol) [0x00007FF69F09BF2C]
	(No symbol) [0x00007FF69F0C7BBF]
	(No symbol) [0x00007FF69F09BEDD]
	(No symbol) [0x00007FF69F09BDAD]
	(No symbol) [0x00007FF69F0C7E60]
	(No symbol) [0x00007FF69F09BEDD]
	(No symbol) [0x00007FF69F0E1B07]
	(No symbol) [0x00007FF69F0C77E3]
	(No symbol) [0x00007FF69F09B3EC]
	(No symbol) [0x00007FF69F09A8CD]
	(No symbol) [0x00007FF69F09AFB1]
	Microsoft::Applications::Events::Event